[View in Colaboratory](https://colab.research.google.com/github/central-ldn-data-sci/advent_of_code/blob/master/2018/python/2018_solutions.ipynb)

## Advent of Code 2018

Festive greetings. Last meetup we looked at the [advent of code 2017](https://adventofcode.com/2017). Advent of Code is a series of small programming puzzles for a variety of skill sets and skill levels in any programming language you like. People use them as a speed contest, interview prep, company training, university coursework, practice problems, or to challenge each other.

There is a new challenge posted each day between 1st - 25th December. These challenges are very accessible to begin with and then they get a bit harder each day. 

To help keep the festive spirit we hope to post solutions to this notebook for each of the 2018 challenges. So check back each day for solutions. 


---

## The challenges

For each challenge you will be given an input file. The input file that you are generated by the Advent of Code is specific to your user account. As a result the answers generated below will likely not be correct for your Advent of Code account - so no cheating and just copying these answers!

The following bsah script will download the files from the github to where you have forked/collab'd this notebook too. 

In [ ]:
%%bash
for i in {1..25};
do
  url=https://raw.githubusercontent.com/central-ldn-data-sci/advent_of_code/master/2018/python/input$i.txt
  if [[ `wget -S --spider $url 2>&1 | grep 'HTTP/1.1 200 OK'` ]];
  then 
    wget $url; 
  fi
done

In [80]:
# packages
import numpy as np
import pandas as pd
import re

### Day 1:

In [82]:
# %%writefile oj-day1.py
import numpy as np

data = np.loadtxt('input1.txt')
print(data.sum())

s = set([0]) # starts with freq. 0
m = 0
s2 = set(data.cumsum() + m)

while(s.isdisjoint(s2)): # returns TRUE if the set has no elements in common with s2
    s.update(data.cumsum() + m)
    m = data.sum() + m
    s2 = set(data.cumsum() + m)
print(tuple(s.intersection(s2))[0])

569.0
77696.0


### Day 2:

In [83]:
# %%writefile oj-day2.py
import numpy as np
from collections import Counter # Counter gives a table of the occurrences in the object passed.

data = np.loadtxt('input2.txt', dtype = "str")

two_sum = 0
three_sum = 0

for d in data:
    if (2 in Counter(d).values()): two_sum += 1 # for numpy strings counter tables the letters
    if (3 in Counter(d).values()): three_sum += 1

print(two_sum * three_sum)

def hamming(str1, str2):
    assert len(str1) == len(str2)
    return sum(ch1 != ch2 for ch1,ch2 in zip(str1,str2))

def common_lets(data):
    l = len(data)
    for i, d in enumerate(data):
        iter = 1
        while(i + iter < l):
            if(hamming(d, data[iter]) == 1):
                return("".join(ch1 for ch1,ch2 in zip(d,data[iter]) if ch1==ch2))
            iter += 1

print(common_lets(data))

5976
xretqmmonskvzupalfiwhcfdb


### Day 3:

In [84]:
# %%writefile oj-day3.py
import codecs
import numpy as np
import re

data = codecs.open('input3.txt', 'r', encoding='utf-8').readlines()
print(data[0])

# check max grid size
max_x = 0
max_y = 0
for d in data:
    y = int(re.match(r'#\d+\s@\s(.*),', d)[1])
    y2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s(.*)x', d)[1])
    max_y = y+y2 if y+y2 > max_y else max_y
    x = int(re.match(r'#\d+\s@\s\d+,(.*):', d)[1])
    x2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s\d+x(.*)$', d)[1])
    max_x = x+x2 if x+x2 > max_x else max_x


# make numpy grid
grid = np.zeros((max_x+1, max_y+1))
for i, d in enumerate(data):
    y = int(re.match(r'#\d+\s@\s(.*),', d)[1])
    y2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s(.*)x', d)[1])
    x = int(re.match(r'#\d+\s@\s\d+,(.*):', d)[1])
    x2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s\d+x(.*)$', d)[1])
    grid[x:x+x2, y:y+y2] += 1;
print(np.sum(grid > 1))
    
# check for completeness
for i, d in enumerate(data):
    y = int(re.match(r'#\d+\s@\s(.*),', d)[1])
    y2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s(.*)x', d)[1])
    x = int(re.match(r'#\d+\s@\s\d+,(.*):', d)[1])
    x2 = int(re.match(r'#\d+\s@\s\d+,\d+:\s\d+x(.*)$', d)[1])
    if np.all(grid[x:x+x2, y:y+y2] == 1):
        print(i)
        break
  

#1 @ 604,670: 22x16

110389
551


### Day 4:

In [85]:
# %%writefile oj-day4.py
import codecs
import numpy as np
import re

data = codecs.open('input4.txt', 'r', encoding='utf-8').readlines()

# grab our times
times = []
for time in data:
    times.append([i for i in re.findall("[\d]+|wakes up|falls asleep", time)])
    
# sort our times
def abs_time(elem):
    return elem[0]*365*24*60 + elem[1]*30*24*60 + elem[2]*24*60 + elem[3]*60 + elem[4]

times.sort(key = abs_time) 

# create our dict of guard snoozes
snoozes = {}
for i, time in enumerate(times):
    year, month, day, hour, minute, action = time
    if action.isdigit():
        if action not in snoozes:
            snoozes[action] = np.zeros(60)
        guard = action
    if action == 'falls asleep':
        snoozes[guard][int(minute):int(times[i+1][4])] += 1

# then for part one we want to find the snooziest guard
sorted_snoozes = sorted(snoozes.items(), key = lambda e: np.sum(e[1]), reverse=True)
print(int(sorted_snoozes[0][0]) * sorted_snoozes[0][1].argmax())

# and for part two we want to find the guard with the most snoozy minute
sorted_snoozes = sorted(snoozes.items(), key = lambda e: np.max(e[1]), reverse=True)
print(int(sorted_snoozes[0][0]) * sorted_snoozes[0][1].argmax())

142515
5370


### Day 5

In [87]:
# %%writefile oj-day5_quicker.py
import codecs

def invert_case(char):
    return (char.lower() if char.isupper() else char.upper())

def remove_values_from_list(l, val):
    return [value for value in l if value not in val]

data = codecs.open('input5.txt', 'r', encoding='utf-8').readlines()
#data = list("dabAcCaCBAcCcaDA")

def remove_copies(data):
    cpy = data[:] # we don't need this in a function, but would outside as python does a shallow copy
    while("!" in cpy or len(cpy)==len(data)):
        cpy = remove_values_from_list(cpy, "!")
        invert = [invert_case(c) for c in cpy]
        for i in range(len(cpy)-1):
            if (cpy[i] != '!') and (cpy[i] == invert[i+1]):
                cpy[i:(i+2)] = '!'*2 # python will not iterate and replace each with !, we have to create that
    return(cpy)

print(len(remove_copies(list(data[0]))))  

lowers = set(str(data[0]).lower())
print(min([len(remove_copies(remove_values_from_list(data[0], {l, invert_case(l)}))) for l in lowers]))

10250
6188


### Day 6

In [88]:
# %%writefile oj-day6.py
from collections import defaultdict

def part1(lines):
    coords = set()
    max_x = max_y = 0

    for line in lines:
        x, y = map(int, line.split(", "))
        coords.add((x, y))
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    coord_id_to_point = {coord_id: point for coord_id, point in enumerate(coords, start=1)}
    region_sizes = defaultdict(int)
    infinite_ids = set()

    for i in range(max_x + 1):
        for j in range(max_y + 1):
            min_dists = sorted([(abs(x - i) + abs(y - j), coord_id) for coord_id, (x, y) in coord_id_to_point.items()])

            if len(min_dists) == 1 or min_dists[0][0] != min_dists[1][0]:
                coord_id = min_dists[0][1]
                region_sizes[coord_id] += 1

                if i == 0 or i == max_x or j == 0 or j == max_y:
                    infinite_ids.add(coord_id)

    return max(size for coord_id, size in region_sizes.items() if coord_id not in infinite_ids)


def part2(lines, manhattan_limit=10000):
    coords = set()
    max_x = max_y = 0

    for line in lines:
        x, y = map(int, line.split(", "))
        coords.add((x, y))
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    size_shared_region = 0

    for i in range(max_x + 1):
        for j in range(max_y + 1):
            size_shared_region += int(sum(abs(x - i) + abs(y - j) for x, y in coords) < manhattan_limit)

    return size_shared_region


lines = [line.strip() for line in open("input6.txt", "r").readlines()]
print(part1(lines))
print(part2(lines))

3290
45602


### Day 7

In [89]:
#%%writefile oj-day7.py
import re
data = [line.strip() for line in open("input7.txt", "r").readlines()]

# just for printing our list
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

## Part 1
def part_one():
    
    # grab our letters
    letters = []
    for d in data:
        letters.append([i for i in re.findall("Step ([\w]) must be finished before step ([\w]) can begin", d)][0])
    starts = [s[0] for s in letters]
    ends = [s[1] for s in letters]
    all = set(starts + ends)

    # make our order
    order = []
    while len(order) < 26:
        starts = [s[0] for s in letters]
        ends = [s[1] for s in letters]
        not_end = [a for a in all if a not in set(ends)]
        order.append(sorted(not_end)[0])
        all.remove(order[-1])
        letters = [l for l in letters if l[0] not in order]

    print(concatenate_list_data(order))
    return(order)

## Part 2

import numpy as np

def can_build(letter, letters, new_order):
    depends = set([s[0] for s in letters if s[1] == letter])
    if depends.issubset(set(new_order)):
        return True
    else:
        return False

def give_job(letter, workers, t, numbered, next_event):
    i = 0
    assigned = False
    while not assigned:
        if workers[i][0] == "-":
            workers[i][0] = letter
            workers[i][1] = t + 59 + numbered[letter]
            assigned = True
        else:
            i += 1
            
def clear_worker(workers, next_event, new_order):
    cleared = False
    for i in range(5):
        if workers[i][1] == next_event and workers[i][1] != float('Inf'):
            new_order.append(workers[i][0])
            workers[i][0] = "-"
            workers[i][1] = float('Inf')
            cleared = True
    if cleared:
        next_event += 1
    else:
        next_event = float('Inf')
        for i in range(5):
            if type(workers[i][1]) == int:
                if workers[i][1] < next_event:
                    next_event = workers[i][1]
    return(next_event)

def part_two(order):
    
    numbered = {a: i for i, a in enumerate(sorted(order), 1)}
    new_order = []
    workers = {a: ["-", float('Inf')] for a in range(5)}

    letters = []
    for d in data:
        letters.append([i for i in re.findall("Step ([\w]) must be finished before step ([\w]) can begin", d)][0])
    starts = [s[0] for s in letters]
    ends = [s[1] for s in letters]

    t = 0
    next_event = float('Inf')

    while len(new_order) < 26:  
        to_clear = []
        for o in order:
            if np.any([True for value, t in workers.values() if value == "-"]):
                if can_build(o, letters, new_order):
                    give_job(o, workers, t, numbered, next_event)
                    to_clear.append(o)

        next_event = min([w[1] for w in workers.values()])
        order = [o for o in order if o not in to_clear]
        next_event = clear_worker(workers, next_event, new_order)
        t = next_event

    print(t)       
 
order = part_one()
part_two(order)

BHMOTUFLCPQKWINZVRXAJDSYEG
877


### Day 8

In [93]:
# %%writefile oj-day8.py
import numpy as np

data = np.loadtxt('input8.txt', dtype=int)

def part_one(data):
    children, metas = data[:2]
    data = data[2:]
    scores = []
    totals = 0

    for i in range(children):
        total, score, data, scoring = part_one(data)
        totals += total
        scores.append(score)

    totals += sum(data[:metas])

    if children == 0:
        return (totals, sum(data[:metas]), data[metas:], scores)
    else:
        return (
            totals,
            sum(scores[k - 1] for k in data[:metas] if k > 0 and k <= len(scores)),
            data[metas:],
            scores
        )

total, value, remaining, scores = part_one(data)

print(total)
print(value)

45868
19724


### Day 9

In [94]:
# %%writefile oj-day9.py
from collections import deque
import re

def part_one(num_players, max_number):
    scores = {}
    game = deque([0])

    for i in range(1, max_number + 1):
        player = i % num_players
        if i % 23 == 0:
            game.rotate(7)
            if player in scores.keys():
                scores[player] += i + game.pop()
            else:
                scores[player] = i + game.pop()
            game.rotate(-1)
        else:
            game.rotate(-1)
            game.append(i)

    return max(scores.values())

data = [line.strip() for line in open("input9.txt", "r").readlines()]
print(part_one(int(re.findall("[\d]+", data[0])[0]),
          int(re.findall("[\d]+", data[0])[1])))
print(part_one(int(re.findall("[\d]+", data[0])[0]),
          int(re.findall("[\d]+", data[0])[1])*100))

384475
3187566597


### Day 10

In [95]:
# %%writefile oj-day10.py
import re
import numpy as np
data = [line.strip() for line in open("input10.txt", "r").readlines()]

coords = []
min_x = min_y = max_x = max_y = 0
for i, d in enumerate(data):
    coords.append([int(r) for r in re.findall(".[\d]+", d)])
 
for i in range(20000):
    cs = [(c[0] + (i*c[2]), c[1] + i*c[3]) for c in coords]
    min_x = min([x[0] for x in cs])
    max_x = max([x[0] for x in cs])
    min_y = min([x[1] for x in cs])
    max_y = max([x[1] for x in cs])
    height = max_y - min_y + 1
    width = max_x - min_x + 1
    if(height) < 30:
        cs = [(c[0] - min_x, c[1]) for c in cs]
        cs = [(c[0], c[1] - min_y) for c in cs]
        grid = np.chararray((height,width), unicode = True)
        grid[:] = "."
        for c in cs: grid[c[1],c[0]] = "x"
        for l in grid: print("".join(l))
        print(i)
        

...........xx......x..........................x...x.....................
....xxx...x.................xx........x.x........................x......
.......xx.............x.....x.....x......xx......x..x.x.............x...
..........x.x.x..xx.....xx.......x....xxx.x...x....x.x....x...x.........
........x.xxxx...x..x.x..x.x.....x.......x....xx....x....x.xxx....xxx...
...............x.x.x......x...x...x..x..xx.................x.....x.x...x
...xxx..xxx.x.x..xx....x.x.xxxxxx..xxx...........xx.................x...
x...x..xx...x.x.x....xx..xxx..x..xx.xx..x.x...x....x...x.x..x...........
..xx.x.xx...x..x.x...xxx.xxxx..x...x.xxx...xxx....xx.x...x..x.xxx.x...x.
....x..x...xx..xx.x.xx....x.x.xx.x.....x.x..x..x....xx.xxx....xx........
..x.x...x.xx.......x.x....x.xx.....x..xxx..x.x.x.x....xx.x..x......x.x..
......xxxx......x..x...x..x.....x.x.....x...x.x..x...xx......xx.........
..x.xx...x.xx...xx...x....x......x...xxx.x.........xx.x..x...xx..x......
...xx....xx.x.........x........x............x......

### Day 11

In [44]:

# part one
grid = np.zeros((300,300))
for i in range(300):
    for j in range(300):
        grid[i,j] x= power_level((i,j),data)

grid_sums = np.zeros((298,298))        
max = 0
for i in range(298):
    for j in range(298):
        grid_sums[i,j] = np.sum(grid[i:i+3,j:j+3])
        

In [51]:
# %%writefile oj-day11.py
import re
import numpy as np
data = int(np.loadtxt("input11.txt"))

def power_level(coord, data):
    
    #Find the fuel cell's rack ID, which is its X coordinate plus 10.
    rack_ID = coord[0] + 10

    #Begin with a power level of the rack ID times the Y coordinate.
    power = rack_ID*coord[1]

    #Increase the power level by the value of the grid serial number (your puzzle input).
    power += data

    #Set the power level to itself multiplied by the rack ID.
    power *= rack_ID

    #Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
    power = int(str(power)[-3]) if power > 99 else 0

    #Subtract 5 from the power level.
    power -= 5
    
    return(power)

# part one
grid = np.zeros((300,300))
for i in range(1,300):
    for j in range(1,300):
        grid[i-1,j-1] = power_level((i,j),data)

grid_sums = np.zeros((298,298))        
max = 0
for i in range(298):
    for j in range(298):
        grid_sums[i,j] = np.sum(grid[i:i+3,j:j+3])
        
print((int(np.argmax(grid_sums) / 298)+1,np.argmax(grid_sums)%298 + 1))     

# part two
maxes = {}
running_max = 0
running_max_k = 0
for k in range(1,30):
    grid_sums = np.zeros((300-k+1,300-k+1))        
    max_k = 0
    for i in range(300-k+1):
        for j in range(300-k+1):
            grid_sums[i,j] = np.sum(grid[i:i+k,j:j+k])
    maxes[k] = (int(np.argmax(grid_sums) / (300-k+1)) + 1,
                int(np.argmax(grid_sums) % (300-k+1)) + 1,
                np.max(grid_sums))  
    if maxes[k][2] > running_max:
        running_max = maxes[k][2]
        running_max_k = k
        
print((maxes[running_max_k][0],maxes[running_max_k][1],running_max_k))

(21, 93)
(231, 108, 14)
